#### 플로우
1. 출발지 입력받기
2. 출발지 위/경도 추출
3. 추천된 것들 중 가장 가까운 곳 지정 및 경로 설정
4. 그 다음으로 가까운 곳 지정 및 경로 설정(반복)
5. 마지막 지점에서 출발지로 다시 경로 설정
6. 지도 표현

In [25]:
# 출발지 입력시 위도, 경도 반환하는 함수
def geocoding(address):
    from geopy.geocoders import Nominatim
    geo_local = Nominatim(user_agent='South Korea')
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y
    except:
        return [0,0]


def Route_Navigation(df, start_address):
    #패키지 임포트
    import pandas as pd
    import networkx as nx
    import webbrowser
    import osmnx as ox
    import matplotlib.cm as cm
    import matplotlib.colors as colors
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import matplotlib.font_manager as fm
    import folium
    import geopandas as gpd
    import json
    import warnings
    import requests
    from geopy.geocoders import Nominatim
    from folium import Marker

    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)

    #출발지점 좌표 추출하기
    startad = geocoding('백현로 100')
    #추천된 지명들 가져오기
    recom = df
    # 시리즈를 데이터프레임으로 변환 후 기존 데이터프레임에 추가
    new_row = pd.Series({'VISIT_AREA_NM':'출발지','X_COORD':startad[0], 'Y_COORD':startad[1]})
    recom = pd.concat([new_row.to_frame().T, recom], ignore_index=True)
    recom_gdf = gpd.GeoDataFrame(recom, geometry=gpd.points_from_xy(recom.Y_COORD, recom.X_COORD), crs='epsg:4326')

    #출발지로부터 최단거리인 다음 행선지 추출
    example_distances = recom_gdf.to_crs(epsg=5186)[:].geometry.distance(recom_gdf.to_crs(epsg=5186).iloc[0].geometry)
    example_dist_sorted = example_distances.sort_values().index
    start_idx = example_dist_sorted[0]
    end_idx = example_dist_sorted[1]
    
    #가장 가까운 지역 정보 추출
    start = recom_gdf.iloc[start_idx][['X_COORD','Y_COORD']]
    end = recom_gdf.iloc[end_idx][['X_COORD','Y_COORD']]
    
    #Folium 객체 생성
    m = folium.Map(location=start, zoom_start=10)
    #최단경로 탐색 위한 OXMnx 객체 생성
    G = ox.graph_from_point(start, dist=50000, network_type='drive')
    
    # 가장 가까운 노드 찾기
    orig_node = ox.distance.nearest_nodes(G, X=start[1], Y=start[0])
    dest_node = ox.distance.nearest_nodes(G, X=end[1], Y=end[0])
        
    # 최단 경로 계산
    shortest_route = nx.shortest_path(G, orig_node, dest_node, weight='length')
        
    # 경로의 좌표 추출
    shortest_route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_route]
        
    # 지도에 경로 입력
    folium.PolyLine(locations=shortest_route_coords, color='blue').add_to(m)
    print(f"{recom_gdf.loc[start_idx]['VISIT_AREA_NM']}부터 {recom_gdf.loc[end_idx]['VISIT_AREA_NM']}까지 경로 생성 완료")

    #####################도착지를 출발지로 변환, 출발지에서 다음 도착지 추출 // 반복##########################3
    
    for _ in range(recom_gdf.shape[0]-2):
        recom_gdf.drop(index= start_idx, inplace = True)
        #방금 도착지인 애를 출발지로 하고 추출
        example_distances = recom_gdf.to_crs(epsg=5186).geometry.distance(recom_gdf.to_crs(epsg=5186).loc[end_idx].geometry)
        example_dist_sorted = example_distances.sort_values().index
        start_idx = example_dist_sorted[0]
        end_idx = example_dist_sorted[1]
    
        start = recom_gdf.loc[start_idx][['X_COORD','Y_COORD']]
        end = recom_gdf.loc[end_idx][['X_COORD','Y_COORD']]
    
        # 가장 가까운 노드 찾기
        orig_node = ox.distance.nearest_nodes(G, X=start[1], Y=start[0])
        dest_node = ox.distance.nearest_nodes(G, X=end[1], Y=end[0])
        
        # 최단 경로 계산
        shortest_route = nx.shortest_path(G, orig_node, dest_node, weight='length')
        
        # 경로의 좌표 추출
        shortest_route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_route]
        
        # 지도 생성
        folium.PolyLine(locations=shortest_route_coords, color='blue').add_to(m)
        print(f"{recom_gdf.loc[start_idx]['VISIT_AREA_NM']}부터 {recom_gdf.loc[end_idx]['VISIT_AREA_NM']}까지 경로 생성 완료")
    
    start = recom_gdf.loc[start_idx][['X_COORD','Y_COORD']]
    end = recom_gdf.loc[end_idx][['X_COORD','Y_COORD']]
    
    # 가장 가까운 노드 찾기
    orig_node = ox.distance.nearest_nodes(G, X=end[1], Y=end[0])
    dest_node = ox.distance.nearest_nodes(G, X=startad[1], Y=startad[0])
        
    # 최단 경로 계산
    shortest_route = nx.shortest_path(G, orig_node, dest_node, weight='length')
    
    # 경로의 좌표 추출
    shortest_route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_route]
      
    # 지도 생성
    folium.PolyLine(locations=shortest_route_coords, color='blue').add_to(m)
    print(f"{recom_gdf.loc[end_idx]['VISIT_AREA_NM']}부터 출발지까지 경로 생성 완료")
    #지점 시각화
    for _, row in recom.iterrows():
        Marker(location = [row['X_COORD'], row['Y_COORD']],
               popup=row['도서관명'],
              ).add_to(m)
    m.save('map.html')
    
    # 저장한 HTML 파일을 웹 브라우저에서 열기
    webbrowser.open('map.html')

In [23]:
Route_Navigation('백현로 100')

출발지부터 병점도서관까지 경로 생성 완료
병점도서관부터 두빛나래어린이도서관도서관까지 경로 생성 완료
두빛나래어린이도서관도서관부터 동탄복합문화센터 도서관까지 경로 생성 완료
동탄복합문화센터 도서관부터 태안도서관까지 경로 생성 완료
태안도서관부터 정남도서관까지 경로 생성 완료
정남도서관부터 봉담도서관까지 경로 생성 완료
봉담도서관부터 남양도서관까지 경로 생성 완료
남양도서관부터 송산도서관까지 경로 생성 완료
송산도서관부터 삼괴도서관까지 경로 생성 완료
삼괴도서관부터 양감작은도서관도서관까지 경로 생성 완료
양감작은도서관도서관부터 출발지까지 경로 생성 완료
